## transformersライブラリでT5を試す

ref. https://medium.com/askdata/train-t5-for-text-summarization-a1926f52d281

日本語モデルをロード  
https://huggingface.co/sonoisa/t5-base-japanese

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
  
tokenizer = T5Tokenizer.from_pretrained("sonoisa/t5-base-japanese")
# model = AutoModelWithLMHead.from_pretrained("sonoisa/t5-base-japanese")
model = T5ForConditionalGeneration.from_pretrained('sonoisa/t5-base-japanese')

In [9]:
' '.join(tokenizer.tokenize('6日、荒川静香が東京ミッドタウンのスケートリンクのセレモニーに出席した。'))

'▁6 日 、 荒川 静 香 が 東京 ミッド タウン の スケート リンク の セレモニー に出席し た 。'

### データセットの準備

In [5]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': ['train.csv'], 'test': 'test.csv'})
train_dataset = dataset['train']
test_dataset = dataset['test']

Using custom data configuration default-ad8ccb2b621f9de2
Reusing dataset csv (/home/sugimoto/.cache/huggingface/datasets/csv/default-ad8ccb2b621f9de2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [8]:
tokenizer.tokenize(test_dataset[0]['summary'])

['▁',
 '新宿',
 '駅構内',
 'で',
 '31',
 '日まで',
 '、「',
 '新宿',
 'ドラゴンクエスト',
 'ジャック',
 '」',
 'が開催される',
 '。',
 '「',
 'ブロック',
 'モンスター',
 '討伐',
 '作戦',
 '」「',
 '新宿',
 'モンスター',
 'ロード',
 '」',
 'の',
 '2',
 'つ',
 'を',
 '楽しめる',
 '。',
 '初日',
 'は約',
 '18',
 '万',
 '個の',
 'ブロック',
 'で',
 'ドット',
 '状に',
 '描いた',
 'モンスター',
 'が',
 '出現する',
 'そう']

In [24]:
def preprocess(text):
    text = text.lower()
    text = text.replace('\n', '')
    return text

def tokenize(batch):
    text = list(map(preprocess,  batch['text']))
    summary = list(map(preprocess, batch['summary']))
    # 大文字を小文字に変換しておく
    tokenized_input = tokenizer(text, padding='max_length', truncation=True, max_length=500)
    tokenized_label = tokenizer(summary, padding='max_length', truncation=True, max_length=100)
    tokenized_input['labels'] = tokenized_label['input_ids']
    return tokenized_input

In [25]:
# tokenizer(['暑い夏の日'], return_tensors="pt")

In [26]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=512)
# test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Loading cached processed dataset at /home/sugimoto/.cache/huggingface/datasets/csv/default-ad8ccb2b621f9de2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-cfd154398e4c6ed0.arrow


In [27]:
model = model.cuda()

In [28]:
import torch
input_ids = torch.LongTensor(train_dataset['input_ids'][:5]).cuda()

In [29]:
decoded_ids = model.generate(input_ids)

In [30]:
decoded_tokens = tokenizer.convert_ids_to_tokens(decoded_ids[0])
eos_idx = decoded_tokens.index('</s>')
# decoded_tokens[1: eos_idx]

In [31]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='.',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_name', # Wandb run name
    logging_steps=1000, # How often to log loss to wandb
    eval_steps=1000, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=470, training_loss=2.475035062749335)

In [36]:
model.config.max_length

512

In [37]:
text = "任天堂の据え置き型ゲーム機「wii u」に年内生産終了報道が出ている。これは3月23日付けの日経新聞 電子版が伝えているもの。「wii u」は発売からまだ3年強だが、「（任天堂は）販売が振るわず、回復が見込めないと判断した」（同紙より）という。「wii u」を巡ってはここ最近、「スーパーマリオメーカー」や「splatoon（スプラトゥーン）」のヒットや、世界的なヒット作「マインクラフト」の投入などにより「本体が欲しい」とのニーズが高まる一方、ネットでは「お店に売っていない」「どこで買えるの？」など品薄状態への不満の声が上がっていた。"
inputs = tokenizer.encode(text, padding='max_length', return_tensors="pt", max_length=500, truncation=True).cuda()

In [38]:
correct = '任天堂の据え置き型ゲーム機「Wii U」に年内生産終了報道が出ている。「（任天堂は）販売が振るわず、回復が見込めないと判断した」という。一方で「どこで買えるの？」など品薄状態への不満の声も上がっていた'

In [39]:
import torch

# model.eval()
with torch.no_grad():
    summary_ids = model.generate(inputs) #, max_length=512, min_length=5, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    print(summary)

<pad> 任天堂の据え置き型ゲーム機「wii u」に年内生産終了が報じられた。「お店に売っていない」「どこで買えるの?」などの不満の声が上がっていたそう。ネットでは「どこで買えるの?」など品薄状態への不満の声が上がっていた</s>


In [ ]:
torch.

### 考察

tokenizationにおいてUNKが多く発生する  
特に大文字英単語はほとんどカバーできていない印象

In [58]:
print(tokenizer.tokenize('Wii U'))
print('W' in tokenizer.get_vocab())
print('Ｗ' in tokenizer.get_vocab())

['W', '▁ii', '▁', 'U']
True
False


In [29]:
tokenizer.vocab_files_names

{'vocab_file': 'spiece.model'}

In [31]:
import os
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load(os.path.join(tokenizer.vocab_file))

True

In [37]:
text = '任天堂の据え置き型ゲーム機「Wii U」に年内生産終了報道が出ている。'
tokens = sp.EncodeAsPieces(text)
# print(tokenizer.tokenize(text))
token_ids = sp.EncodeAsIds(text)

In [38]:
sp.DecodeIds(token_ids)

'任天堂の据え置き型ゲーム機「 ⁇ ii  ⁇ 」に年内生産終了報道が出ている。'

In [50]:
sp['W'], sp['Ｗ'], sp['３'], sp['3'], sp['w'], sp['ｗ']

(2, 2, 2, 25, 396, 2)

In [43]:
sp.IdToPiece(2)

'<unk>'

In [25]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('/home/sugimoto/japanese_bart_base_2.0')

OSError: Can't load tokenizer for '/home/sugimoto/japanese_bart_base_2.0'. Make sure that:

- '/home/sugimoto/japanese_bart_base_2.0' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/home/sugimoto/japanese_bart_base_2.0' is the correct path to a directory containing relevant tokenizer files



In [24]:
tokenizer.tokenize("放 射線 治療", do_basic_tokenize=False)

Keyword arguments {'do_basic_tokenize': False} not recognized.


['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']